In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic  = mp.solutions.holistic #holistic model
mp_drawing = mp.solutions.drawing_utils #utilidades de dibujo

In [4]:
def mediapipe_detection(image, model):
    # Convert from BGR to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Make a copy of the image to avoid modifying the original image
    input_image = np.copy(image_rgb)
    
    # Make the input image non-writable before processing
    input_image.flags.writeable = False
    
    # Perform detection
    results = model.process(input_image)
    
    # Make the input image writable again
    input_image.flags.writeable = True
    
    # Convert the image back from RGB to BGR
    image = cv2.cvtColor(input_image, cv2.COLOR_RGB2BGR)
    
    return image, results



In [5]:
def draw_landmarks(image,results): #dibujar las marcas en la imagen
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION) #dibuja las conecciones de la cara
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS) #Dibuja las pose conecciones
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS) #dibuja las conecciones de la mano izquierda
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS) #dibuja las conecciones de la mano derecha 

    

In [6]:
def draw_styled_landmarks(image,results): #dibuja los landmarks refinados 
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION,mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),mp_drawing.DrawingSpec(color=(80,256,121),thickness=1,circle_radius=1)) #dibuja las conecciones de la cara
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,mp_drawing.DrawingSpec(color=(80,22,10),thickness=2,circle_radius=4),mp_drawing.DrawingSpec(color=(80,44,121),thickness=2,circle_radius=2)) #Dibuja las pose conecciones
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,mp_drawing.DrawingSpec(color=(121,22,76),thickness=2,circle_radius=4),mp_drawing.DrawingSpec(color=(121,44,250),thickness=2,circle_radius=2)) #dibuja las conecciones de la mano izquierda
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=4),mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2)) #dibuja las conecciones de la mano derecha 


In [7]:
#abre la webcam y va entre los frames
cap = cv2.VideoCapture(0)
#setteamos el modelo de mediapipe
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
  while cap.isOpened():
    #lee el feed
    ret,frame = cap.read()


    #hace la deteccion
    image, results = mediapipe_detection(frame,holistic)
    print(results)
    #dibuja  los landmarks
    draw_styled_landmarks(image,results)
    #Lo muestra en la pantalla
    cv2.imshow('OpenCV Feed',image)

    #Rompe el loop si se apreta q
    if cv2.waitKey(10) & 0xFF == ord('q'):
      break
  cap.release()
  cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [8]:
pose = [] #array para gaursar los landmarks
for res in results.pose_landmarks.landmark:
    test = np.array([res.x,res.y,res.z,res.visibility])
    pose.append(test)

In [9]:
pose = np.array([[res.x,res.y,res.z,res.visibility]for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)#todos los landmarks en un solo array grande
face = np.array([[res.x,res.y,res.z]for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)#todos los landmarks en un solo array grande
lh = np.array([[res.x,res.y,res.z]for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)#todos los landmarks en un solo array grande
rh = np.array([[res.x,res.y,res.z]for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)#todos los landmarks en un solo array grande

In [10]:

def extract_keypoints(results):
    pose = np.array([[res.x,res.y,res.z,res.visibility]for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)#todos los landmarks en un solo array grande
    face = np.array([[res.x,res.y,res.z]for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)#todos los landmarks en un solo array grande
    lh = np.array([[res.x,res.y,res.z]for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)#todos los landmarks en un solo array grande
    rh = np.array([[res.x,res.y,res.z]for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)#todos los landmarks en un solo array grande
    return np.concatenate([pose,face,lh,rh])
    

In [11]:


# Define la ruta a tu carpeta de videos y la ruta de salida para los fotogramas
source_videos_path = 'D:\Codes\EFBA\DataEfba'
frames_output_path = 'D:\Codes\EFBA\FFrames'

# Asegúrate de que la carpeta de salida existe
if not os.path.exists(frames_output_path):
    os.makedirs(frames_output_path)

# Lista todos los archivos en la carpeta de videos
video_files = [f for f in os.listdir(source_videos_path) if f.endswith('.mp4')]

for video_file in video_files:
    video_path = os.path.join(source_videos_path, video_file)
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Define el nombre del archivo de salida para el fotograma
        frame_filename = os.path.join(frames_output_path, f"{video_file}_frame_{frame_count}.png")
        
        # Guarda el fotograma como imagen PNG
        cv2.imwrite(frame_filename, frame)
        
        frame_count += 1

    cap.release()


In [12]:

# Ruta a la carpeta con los fotogramas extraídos
frames_path = 'D:\Codes\EFBA\FFrames'
# Ruta a la carpeta donde guardarás los fotogramas procesados
processed_frames_path = 'D:\Codes\EFBA\PFrames'

# Asegúrate de que la carpeta de destino existe
if not os.path.exists(processed_frames_path):
    os.makedirs(processed_frames_path)

# Tamaño al que redimensionarás los fotogramas
new_size = (128, 128)

for frame_name in os.listdir(frames_path):
    frame_path = os.path.join(frames_path, frame_name)
    # Carga el fotograma
    frame = cv2.imread(frame_path)
    # Redimensiona el fotograma
    resized_frame = cv2.resize(frame, new_size)
    # Normaliza los valores de los píxeles
    normalized_frame = resized_frame / 255.0
    # Guarda el fotograma procesado (si es necesario, o puedes mantenerlo en memoria para el entrenamiento directamente)
    processed_frame_path = os.path.join(processed_frames_path, frame_name)
    # Para guardar, es necesario revertir la normalización (para fines de visualización)
    cv2.imwrite(processed_frame_path, normalized_frame * 255)



In [13]:
tags = {'allen': 0 ,
        'bird': 1,
        'curry': 2,
        'harden': 3,
        'jj': 4,
        'jordan': 5,
        'kd': 6,
        'kerr': 7,
        'klay': 8,
        'kobe': 9,
        'korver': 10,
        'lillard':11,
        'miller':12}
        

In [14]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [15]:



data_route = 'D:\Codes\EFBA\OGFrames'

# Preparar listas para almacenar los datos y las etiquetas
X = []  # Datos (fotogramas)
y = []  # Etiquetas

# Recorrer cada carpeta y cargar los fotogramas
for tag, value in tags.items():
    folder = os.path.join(data_route, tag)
    for archivo in os.listdir(folder):
        archive_route = os.path.join(folder, archivo)
        
        # Carga y preprocesa el fotograma aquí (por ejemplo, redimensionar)
        frame = cv2.imread(archive_route)
        frame = cv2.resize(frame, (128, 128))  # Ejemplo de redimensionamiento
        frame = frame / 255.0  # Normalización
        
        # Añade el fotograma y su etiqueta a las listas
        X.append(frame)
        y.append(value)

# Convertir a arrays de NumPy y codificar las etiquetas como one-hot
X = np.array(X)
y = to_categorical(np.array(y))

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [16]:
label_map = {label:num for num,label in enumerate(video_file)}

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv2D, MaxPooling2D, Flatten, TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2

In [35]:
model = Sequential()

# Capa TimeDistributed para procesar cada fotograma de la secuencia
# Asume que cada fotograma ha sido redimensionado a 128x128 y tiene 3 canales (RGB)
model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(None, 128, 128, 3)))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Flatten()))

# Capas LSTM para procesar la secuencia de vectores obtenidos de los fotogramas
model.add(LSTM(64, return_sequences=True, activation='relu'))
model.add(LSTM(32, return_sequences=False, activation='relu'))

# Capas densas para clasificación final
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
# Asume que hay 3 clases en tu clasificación. Ajusta según el número real de clases.
model.add(Dense(3, activation='softmax'))

# Resumen del modelo
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_3 (TimeDi  (None, None, 126, 126,    896       
 stributed)                  32)                                 
                                                                 
 time_distributed_4 (TimeDi  (None, None, 63, 63, 32   0         
 stributed)                  )                                   
                                                                 
 time_distributed_5 (TimeDi  (None, None, 127008)      0         
 stributed)                                                      
                                                                 
 lstm_2 (LSTM)               (None, None, 64)          32530688  
                                                                 
 lstm_3 (LSTM)               (None, 32)                12416     
                                                      

In [36]:
num_frames = 125  # Número de fotogramas por video

# Calcular el nuevo número de "videos" para entrenamiento y prueba, respectivamente
total_fotogramas_train = X_train.shape[0]
total_fotogramas_test = X_test.shape[0]

nuevo_numero_de_videos_train = total_fotogramas_train // num_frames
nuevo_numero_de_videos_test = total_fotogramas_test // num_frames

# Calcular el nuevo total de fotogramas para asegurarse de que cada secuencia tenga exactamente num_frames fotogramas
nuevo_total_fotogramas_train = nuevo_numero_de_videos_train * num_frames
nuevo_total_fotogramas_test = nuevo_numero_de_videos_test * num_frames

# Recortar X_train y X_test para ajustarse al nuevo número total de fotogramas
X_train_cortado = X_train[:nuevo_total_fotogramas_train]
X_test_cortado = X_test[:nuevo_total_fotogramas_test]

# Reestructurar X_train y X_test para reflejar esta estructura
X_train_reshaped = X_train_cortado.reshape(nuevo_numero_de_videos_train, num_frames, 128, 128, 3)
X_test_reshaped = X_test_cortado.reshape(nuevo_numero_de_videos_test, num_frames, 128, 128, 3)



In [37]:

# Si y_train está en formato one-hot
num_clases = y_train.shape[1]

# Si y_train está en formato de etiquetas enteras
# num_clases = len(np.unique(y_train))

print(f"Número de clases únicas: {num_clases}")


Número de clases únicas: 13


In [38]:
model.add(Dense(13, activation='softmax'))  # Ajusta el número de unidades a 13

In [39]:
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])


In [32]:
del model

In [40]:
# Ejemplo de entrenamiento, asegúrate de ajustar `epochs` y `batch_size` a tus necesidades
history = model.fit(X_train_reshaped, y_train, epochs=20, batch_size=4, validation_split=0.2)


Epoch 1/20
2/2 [==============================] - 33s 11s/step - loss: 2.7410 - accuracy: 0.0000e+00 - val_loss: 3.0090 - val_accuracy: 0.0000e+00
Epoch 2/20
2/2 [==============================] - 19s 9s/step - loss: 2.7614 - accuracy: 0.0000e+00 - val_loss: 3.0301 - val_accuracy: 0.0000e+00
Epoch 3/20
2/2 [==============================] - 16s 8s/step - loss: 2.5961 - accuracy: 0.1429 - val_loss: 2.9474 - val_accuracy: 0.0000e+00
Epoch 4/20
2/2 [==============================] - 15s 7s/step - loss: 2.5755 - accuracy: 0.1429 - val_loss: 2.9079 - val_accuracy: 0.0000e+00
Epoch 5/20
2/2 [==============================] - 17s 8s/step - loss: 2.7476 - accuracy: 0.0000e+00 - val_loss: 3.0308 - val_accuracy: 0.0000e+00
Epoch 6/20
2/2 [==============================] - 16s 8s/step - loss: 2.6200 - accuracy: 0.1429 - val_loss: 2.9481 - val_accuracy: 0.0000e+00
Epoch 7/20
2/2 [==============================] - 15s 8s/step - loss: 2.8934 - accuracy: 0.0000e+00 - val_loss: 3.0312 - val_accuracy: 

In [48]:
loss, accuracy = model.evaluate(X_test_reshaped, y_test)
print(f"Loss: {loss}, Accuracy: {accuracy}")

ValueError: Data cardinality is ambiguous:
  x sizes: 2
  y sizes: 291
Make sure all arrays contain the same number of samples.

*Real time Testing*

In [ ]:
#Variables de deteccion
sequence = []
sentence = []
predictions = []
threshold = 0.5


#abre la webcam y va entre los frames
cap = cv2.VideoCapture(2)

#setteamos el modelo de mediapipe
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
  while cap.isOpened():
    #lee el feed
    ret,frame = cap.read()

    #hace la deteccion
    image, results = mediapipe_detection(frame,holistic)
    print(results)
    #dibuja  los landmarks
    draw_styled_landmarks(image,results)

    #Logica de prediccion
    keypoints = extract_keypoints(results)
    sequence.append(keypoints)
    #[-30:] son los ultimos 30 frames/ 30 set de keypoints
    sequence = sequence[-30:]
    
    #si la longitud de la sequencia tiene una longitud de 30 frames, recien ahi se hace la prediccion
    if len(sequence) == 30:
        res = model.predict(np.expand_dims(sequence,axis = 0))[0]
        print(actions[np.argmax(res)])
        predictions.append(np.argmax(res))
        
        
    #Visualizacion Logic  
    #if all(res[np.argmax(res)]) > threshold:   
    #if all(res[np.argmax(res)]) > threshold:
    if res[np.argmax(res)] > threshold:
      if len(sentence) > 0:
        if actions[np.argmax(res)] != sentence[-1]:
          sentence.append(actions[np.argmax(res)])
      else:
          sentence.append(actions[np.argmax(res)])
      
    if len(sentence) > 5:
      sentence = sentence[-5:]
    
    #Visualizar probabilidades
    #image = prob_viz(res,actions,image, colors)
    
    cv2.rectangle(image,(0,0),(640,40),(245,117,16),-1)
    cv2.putText(image,' '.join(sentence),(3,30),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
          
    
    #Lo muestra en la pantalla
    cv2.imshow('OpenCV Feed',image)

    #Rompe el loop si se apreta q
    if cv2.waitKey(10) & 0xFF == ord('q'):
      break
  cap.release()
  cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [ ]:
model.predict(x_test[0])